In [ ]:
from __future__ import print_function
import os,sys
import ROOT as rt
from larlite import larlite
from ROOT import larutil
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import lardly

In [ ]:
# LOAD THE FILE
#output_files = ["../example_data/bruce_examples/test_bruceexamples_ssnet.root"]
output_files = ["../test_larlite.root"]
ioll = larlite.storage_manager( larlite.storage_manager.kREAD )
for f in output_files:
    ioll.add_in_filename( f )
ioll.open()
nentries = ioll.get_entries()

In [ ]:
# SETUP THE GEOMETRY

# Set the detector in larlite
#detid = larlite.geo.kICARUS
detid = larlite.geo.kMicroBooNE
#detid = larlite.geo.kSBND
larutil.LArUtilConfig.SetDetector(detid)

# Get geometry class
geo = larlite.larutil.Geometry.GetME()

# Get detector outlines from lardly
from lardly.detectors.getdetectoroutlines import DetectorOutlineFromLarlite
detoutline = DetectorOutlineFromLarlite(detid)
detlines = detoutline.getlines()

In [ ]:
# dump cryo,tpc ids for each plane
for icryo in range(geo.Ncryostats()):
    for itpc in range(geo.NTPCs(icryo)):
        for iplane in range(geo.Nplanes(itpc,icryo)):
            print("(%d,%d,%d): simpleindex=%d"%(icryo,itpc,iplane,geo.GetSimplePlaneIndexFromCTP(icryo,itpc,iplane)))

In [ ]:
# Get entry data
ENTRY = 0
MAX_HITS = 100000
SAMPLED = False
#PLOT_TPC_LIST=[(0,0)]
PLOT_TPC_LIST=[]

ioll.go_to(ENTRY)
ev_hit = ioll.get_data( larlite.data.kLArFlow3DHit, "larmatch")
nhits = ev_hit.size()
orig_nhits = nhits

if nhits>MAX_HITS:
    idx = np.arange(nhits,dtype=np.int32)
    np.random.shuffle(idx)
    SAMPLED = True
    orig_nhits = nhits
    nhits = MAX_HITS
else:
    idx = np.arange(nhits,dtype=np.int32)
    

pos = np.zeros( (nhits,4) )
kplabel = np.zeros( (nhits,6) )

for ihit in range(nhits):
    hitidx = int(idx[ihit])
    hit = ev_hit.at(hitidx)
    ct = (hit.targetwire[5],hit.targetwire[4])
    if len(PLOT_TPC_LIST)>0 and ct not in PLOT_TPC_LIST:
        continue
    for i in range(3):
        pos[ihit,i] = hit[i]
        pos[ihit,3] = hit.track_score
    for i in range(6):
        kplabel[ihit,i] = hit[17+i]

print("pos array: ",pos.shape)

In [ ]:
# Organize Keypoint Reco Data

# define some colors for particle types
kp_color_array = np.array( ((255,0,0),    # nu
                            (0,255,0),    # track-start
                            (0,0,255),    # track-end
                            (255,0,255),  # shower
                            (0,255,255),  # shower-michel
                            (255,255,0)),     # shower-delta
                            dtype=np.float64 ) 

ev_keypoints = ioll.get_data( larlite.data.kLArFlow3DHit, "keypoint" )

npts = ev_keypoints.size()
print("Number of Reco Keypoints: ",npts)
kp_counts = [0,0,0,0,0,0]
for i in range(npts):
    kphit = ev_keypoints.at(i)
    # data in the hit
    # [0-2]: x,y,z
    # [3]: keypoint type
    # [4]: score
    kptype = int(kphit[3])
    kp_counts[kptype] += 1
print("Counts: ",kp_counts)
    
kp_data = {}
for kptype in range(6):
    if kp_counts[kptype]==0:
        continue
    xyz = np.zeros( (kp_counts[kptype], 3), np.float32 )   
    kp_data[kptype] = xyz
    
kp_counts = [0,0,0,0,0,0]

for i in range(npts):
    kphit = ev_keypoints.at(i)
    kptype = int(kphit[3])
    xyz = kp_data[kptype]
    for v in range(3):
        xyz[kp_counts[kptype],v] = kphit[v]
    kp_counts[kptype] += 1
    
kp_traces = []
for kptype in [5,4,3,2,1,0]:
    if kptype not in kp_data:
        continue
    rgb = kp_color_array[kptype]
    xyz = kp_data[kptype]
    zrgb = "rgb(%d,%d,%d)"%(rgb[0],rgb[1],rgb[2])
    plot_trace = {
        "type":"scatter3d",
        "x": xyz[:,0],
        "y": xyz[:,1],
        "z": xyz[:,2],
        "mode":"markers",
        "name":"KP[%d]"%(kptype),
        "marker":{"color":zrgb,"size":5,"opacity":0.8},
    }
    kp_traces.append(plot_trace)

In [ ]:
# PLOT SPACEPOINTS
PLOT_KEYPOINTS = False

SCORE_THRESHOLD = 0.5
xpos = pos[pos[:,3]>SCORE_THRESHOLD,:]

sp_plots = []
print("PLOTTING pos: ",pos.shape)
plot = go.Scatter3d( x=xpos[:,0], y=xpos[:,1], z=xpos[:,2],text=xpos[:,3],
                    mode="markers", name="larmatch", 
                    marker={"size":1.0,"opacity":1.0,
                            "cmin":0.0,
                            "cmax":1.0,
                            "color":xpos[:,3],
                            "colorscale":"Viridis"} )  
sp_plots.append(plot)
print("min score: ",np.min(xpos[:,3]))
print("max score: ",np.max(xpos[:,3]))

if PLOT_KEYPOINTS:
    sp_plots += kp_traces
    
# add detector outline
sp_plots += detlines

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots, layout=layout)
fig.show()
 

In [ ]:
KPINDEX=3
kp_plots = []
plot = go.Scatter3d( x=pos[:,0], y=pos[:,1], z=pos[:,2], 
                    mode="markers", name="larmatch", 
                    marker={"size":1.5,"opacity":1.0,"color":kplabel[:,KPINDEX],"colorscale":"Viridis"} )  
kp_plots.append(plot)

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 2, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=kp_plots+detlines, layout=layout)
fig.show()